In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import ast
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv('/kaggle/input/paper-prophet-data/trainingDATA.csv')
df.head()

In [ ]:
df = df.drop(['year_time'], axis=1)

In [ ]:
df

In [ ]:
df = df[df['categories'].notnull()]

In [ ]:
# finding unique categories
categories = []
for x in df['categories']:
    y = x.split()
    for item in y:
        categories.append(item)

In [ ]:
categories = list(dict.fromkeys(categories))
# df2 = pd.DataFrame(columns=categories)

In [ ]:
# # one-hot-encoding the categories
# for x, idx in zip(df['categories'], range(0, df.shape[0])):
#     y = x.split()
#     row = []
#     for item in df2.columns:
#         if item in y:
#             row.append(1)
#         else:
#             row.append(0)
#     df2.loc[idx] = row

In [ ]:
# df2

In [ ]:
# df3 = pd.concat([df, df2], axis=1).reindex(df2.index)

In [ ]:
# final_df = df3.drop(['categories'], axis=1)

In [ ]:
final_df=pd.read_csv("/kaggle/input/one-hot-encode-2/hot-encoded.csv")

In [ ]:
final_df

In [ ]:
# pre-processing the data
def decontract(sentence):
    # specific
    sentence = re.sub(r"won't", "will not", str(sentence))
    sentence = re.sub(r"can\'t", "can not", sentence)

    # general
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', '', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub("", sentence)

In [ ]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

re_stop_words = re.compile(r"\b(" + "|".join(stopwords) + ")\\W", re.I)

In [ ]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

In [ ]:
final_df['summary'] = final_df['summary'].str.lower()
final_df['summary'] = final_df['summary'].apply(decontract)
final_df['summary'] = final_df['summary'].apply(cleanPunc)
final_df['summary'] = final_df['summary'].apply(keepAlpha)
final_df['summary'] = final_df['summary'].apply(removeStopWords)
final_df['summary'] = final_df['summary'].apply(stemming)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=70000, lower=True)
tokenizer.fit_on_texts(final_df['summary'])
sequences = tokenizer.texts_to_sequences(final_df['summary'])
x = pad_sequences(sequences, maxlen=100)

In [ ]:
final_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    final_df[final_df.columns[2:]], 
                                                    test_size=0.3)

In [ ]:
y_train

In [ ]:
num_classes = y_train.shape[1]
max_words = len(tokenizer.word_index) + 1
maxlen = 512

In [ ]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# pip install tensorflow-addons
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
# import tensorflow as tf
# import tensorflow_addons as tfa  # Import TensorFlow Addons

In [ ]:
# import tensorflow as tf
# import tensorflow_addons as tfa

# # Define the Precision metric
# precision = tfa.metrics.Precision()

In [ ]:
!pip install tensorflow-addons
!pip install tensorflow
!pip install numpy matplotlib seaborn scikit-learn
!pip install tensorflow-addons


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf
import tensorflow_addons as tfa  # Import TensorFlow Addons

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf
import tensorflow_addons as tfa  # Import TensorFlow Addons

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LeakyReLU, Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Slightly modified model architecture
filter_length = 100  # Reduced filter length
maxlen = 100

model = Sequential()
model.add(Embedding(max_words, 50, input_length=maxlen))  # Reduced embedding dimension
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(64, activation='relu'))  # Reduced number of neurons
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])  # Use 'accuracy' as a metric

callbacks = [
    ReduceLROnPlateau(patience=2),  # Reduce learning rate on plateau
    EarlyStopping(patience=5, restore_best_weights=True),  # Early stopping to prevent overfitting
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

history = model.fit(X_train, y_train,
                    epochs=70,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=callbacks)

In [ ]:
model.save('sub_new.h5')

In [ ]:
test = pd.read_csv('/kaggle/input/paper-prophet-test/testingDATA .csv')

In [ ]:
test = test.drop(['year_time'], axis=1)

In [ ]:
test['summary'] = test['summary'].str.lower()
test['summary'] = test['summary'].apply(decontract)
test['summary'] = test['summary'].apply(cleanPunc)
test['summary'] = test['summary'].apply(keepAlpha)
test['summary'] = test['summary'].apply(removeStopWords)
test['summary'] = test['summary'].apply(stemming)

In [ ]:
# tokenizer.fit_on_texts(test['summary'])
sequences = tokenizer.texts_to_sequences(test['summary'])
x = pad_sequences(sequences, maxlen=100)

In [ ]:
preds = model.predict(x)

In [ ]:
predclass=[]
for item in preds:
    predclass.append(np.argmax(item))
predclass

In [ ]:
cnn_model = model
metrics = cnn_model.evaluate(X_test, y_test)
print("{}: {}".format(model.metrics_names[1], metrics[1]))

In [ ]:
classes = []
for item in predclass:
    t = str(categories[item])
    classes.append(t)
classes

In [ ]:
submission = pd.DataFrame({
    'ID': test['ID'],
    'categories': classes
})

In [ ]:
submission.to_csv('sub1.csv', index=False)